<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/09_finetuning_BERT_for_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 感情分析のための日本語BERTのfine-tuning
* 事前学習済みモデルをfine-tuningによってカスタマイズする方法を学ぶ。

* Transformersアーキテクチャについては、以下を参照。
 * https://arxiv.org/abs/1706.03762
* 以下は解説記事。
 * http://jalammar.github.io/illustrated-transformer/
 * http://nlp.seas.harvard.edu/2018/04/03/attention.html

* 今回はHugging FaceのTransformersライブラリからBERTを使う。
 * https://huggingface.co/docs/transformers/
* BERTについては、以下を参照。
 * https://arxiv.org/abs/1810.04805

**ランタイムのタイプはGPUにしておくこと。**

## 必要なライブラリのインストール

In [ ]:
!pip install transformers datasets

### トークナイザを動かすために必要なモジュールのインストール

In [ ]:
!pip install fugashi ipadic

## インポート

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel

np.random.seed(123)
torch.manual_seed(123)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 日本語BERTとしては、今回、下記のモデルを使う。
model_name = "cl-tohoku/bert-base-japanese-whole-word-masking"

## データセット

### WRIME: 主観と客観の感情分析データセット
* https://github.com/ids-cv/wrime

* このデータセットは、元々は-2, -1, 0, ,1 2の５値で感情極性を表現している。
* 今回は、**ネガティブ、ニュートラル、ポジティブの3値**に単純化することにする。
 * 余裕がある方は、5値分類のままファインチューニングを実施してみてください。

In [ ]:
dataset = load_dataset("shunk031/wrime", "ver2")

In [ ]:
print(dataset["train"][0])

* 今回は、ポジティヴ／ニュートラル／ネガティヴの3値分類問題として、感情分析を行なう。
 * ポジティブのラベルを2、ニュートラルのラベルを1、ネガティブのラベルを0と設定する。

In [ ]:
def modify_sentiment_label(example):
  sentiment_dic = {-2:0, -1:0, 0:1, 1:2, 2:2}
  sentiment = example['avg_readers']['sentiment']
  example['label'] = sentiment_dic[sentiment]
  return example

for key in dataset:
  dataset[key] = dataset[key].map(modify_sentiment_label)

* 3つのクラスのサイズを調べてみる。

In [ ]:
from collections import Counter

for key in dataset:
  labels = []
  for example in dataset[key]:
    labels.append(example["label"])
  print(key, Counter(labels))

## トークナイザ

* 日本語BERTのトークナイザをダウンロードしておく。

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
tokenizer

* 試しに、一つのテキストをトークナイズしてみる。

In [ ]:
model_inputs = tokenizer(dataset["train"]["sentence"][0], padding=True, return_tensors="pt")
print(model_inputs)

* `token_type_ids`については、下記URLを参照。
 * https://huggingface.co/docs/transformers/main/en/glossary#token-type-ids

* インデックスの列をトークン列に戻してみる。
 * 先頭と末尾のspecial tokensに注意。

In [ ]:
print(tokenizer.convert_ids_to_tokens(model_inputs['input_ids'][0]))

## DataLoader

* PyTorchのDataLoaderを利用して、ミニバッチをランダムな順で取ってこれるようにする。

* どんなミニバッチにしたいかをcollation用関数で定義する。
 * テキストはトークナイザを通した状態でミニバッチに含ませる。
 * ラベルはPyTorchのTensorに変換してミニバッチに含ませる。

* 今回使っているトークナイザは、テキストの長さが揃っていないときのpaddingの処理までおこなってくれる。

In [ ]:
def collate_fn(batch):
  texts = list()
  labels = list()
  for instance in batch:
    texts.append(instance["sentence"])
    labels.append(instance["label"])
  model_input = tokenizer(texts, padding=True, return_tensors="pt")
  return model_input.to(device), torch.tensor(labels).long().to(device)

* Train/Dev/Testの各スプリットについて、DataLoaderのインスタンスを作成する。
 * ミニバッチのサイズは、チューニングした方が良い。

In [ ]:
BATCH_SIZE = 8
loaders = {}
for key in dataset:
  loaders[key] = DataLoader(
      dataset[key],
      batch_size=BATCH_SIZE,
      shuffle=(key == "train"),
      collate_fn=collate_fn,
      )

* 試しに、一個、ミニバッチを訓練データから取得してみる。

In [ ]:
input, label = next(iter(loaders["train"]))
print(input)
print(label)

## 事前学習済み日本語BERT

### モデルの取得

In [ ]:
bert = AutoModel.from_pretrained(model_name)

In [ ]:
bert

* このBERTの単語埋め込みの次元数を調べる。

In [ ]:
bert.embeddings.word_embeddings.weight.shape

* このBERTのlayer数を調べる。

In [ ]:
len(bert.encoder.layer)

* 最後のlayerだけ表示する。

In [ ]:
bert.encoder.layer[-1]

### pooler
* poolerは、最初のトークンに対応する出力を受け取っている。
* 活性化関数tanhを使っている。
 * https://github.com/huggingface/transformers/blob/e3a4bd2bee212a2d0fd9f03b27fe7bfc1debe42d/src/transformers/models/bert/modeling_bert.py#L654


In [ ]:
bert.pooler

## 分類モデルの定義

* [CLS]トークンに対応する出力だけでなく、全トークンに対応する出力の平均も併せて使うことにする。

In [ ]:
class BERTTextSentiment(nn.Module):
  def __init__(self, bert, num_class):
    super().__init__()
    self.bert = bert
    self.embdim = bert.embeddings.word_embeddings.weight.size(1)
    self.num_class = num_class
    self.fc = nn.Linear(self.embdim * 2, self.num_class)

  def forward(self, input):
    output = self.bert(**input)
    pooler_output = output.pooler_output
    # poolerに合わせて、tanhを適用しておく。
    mean_output = torch.tanh(output.last_hidden_state).mean(1)
    logit = self.fc(torch.cat([pooler_output, mean_output], -1))
    return logit

In [ ]:
num_class = 3
model = BERTTextSentiment(bert, num_class).to(device)

* 最後のlayerと、poolerだけfinetuningするよう、設定する。

In [ ]:
for param in model.bert.parameters():
  param.requires_grad = False
for param in model.bert.encoder.layer[-1].parameters():
  param.requires_grad = True
for param in model.bert.pooler.parameters():
  param.requires_grad = True

* 試しに、ミニバッチを入力してみる。

In [ ]:
input, _ = next(iter(loaders["train"]))
logit = model(input)
print(logit)

## 最適化アルゴリズムと損失関数

* 学習率はチューニングする必要あり。

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

## 訓練を行なう関数

In [ ]:
def train(dataloader):
  model.train()
  total_loss, total_acc, total_count = 0, 0, 0
  for i, batch in enumerate(dataloader):
    input, label = batch
    logit = model(input)
    loss = criterion(logit, label)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    n_instances = label.size(0)
    total_loss += loss.item() * n_instances
    total_acc += (logit.argmax(-1) == label).sum().item()
    total_count += n_instances
    if (i + 1) % 100 == 0:
      print(f"===>{i+1} | acc {total_acc/total_count:.4f}")
  return total_loss / total_count, total_acc / total_count


## 評価を行なう関数

In [ ]:
def evaluation(dataloader):
  model.eval()
  total_loss, total_acc, total_count = 0, 0, 0
  for i, batch in enumerate(dataloader):
    input, label = batch
    with torch.no_grad():
      logit = model(input)
      loss = criterion(logit, label)
    n_instances = label.size(0)
    total_loss += loss.item() * n_instances
    total_acc += (logit.argmax(-1) == label).sum().item()
    total_count += n_instances
    if (i + 1) % 100 == 0:
      print(f"===>{i+1} | acc {total_acc/total_count:.4f}")
  return total_loss / total_count, total_acc / total_count

## fine-tuningの実行

In [ ]:
for epoch in range(1, 6):
  loss, acc = train(loaders["train"])
  dev_loss, dev_acc = evaluation(loaders["validation"])
  print(f'> epoch {epoch} | train loss {loss:.3f} | train acc {acc:.4f} || '
      f'dev loss {dev_loss:.3f} | dev acc {dev_acc:.4f}')

# 本日の課題
* 分類性能が上がるよう、ハイパーパラメータをチューニングしてみよう。
 * 学習率、ミニバッチのサイズ、分類用の全結合層の層数、etc...
* [発展] 今回のデータセットの、元々の5値分類で、fine-tuningを実行してみよう。